# 1. Задача

### Выделить ключевые слова из запроса пользователя в соответствии с иерархической предметной областью.



Ограничения:

1. Слова должны находиться на одной ветке. Приэтом не допускается использование нескольких агрегаторов на **одном уровне.** 
2. Слова, лежащие даже в абсолютно разных ветках, могут быть очень похожи. Например, "ипотека" - "none" -  "агентские", "ипотека" - "агентские обязанности"; "онлайн", "офлайн"; "с господдержкой", "без господдержки"; и т.д.
3. От перестановки слов - ничего не ломается.
4. Допускаются опечатки до некоторого значения.
5. Если есть длинный ключ (), то он хуже обрабатывается базовыми методами, например, расстоянием Левенштейна. Очень маленькие слова также зависят от формы слова (план - планов - низкий Левенштейн).
6. В ключах могут быть уменьшенные синонимы.


# 2. Исходные данные и библиотеки

### Библиотеки

In [1]:
# %pip install requirements_main.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from rapidfuzz import fuzz, process
from sklearn.model_selection import train_test_split

from utils.config import levels
from utils.get_dynamic_query_list import *
from utils.fit_hierarchy import *
from utils.process_query import *

### Исходная иерархия:

In [3]:
with open('data/bank_hierarchy.json', 'r') as f:
    hierarchy = json.load(f)
    
hierarchy

{'Corporate': {'Вклады': {'None': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']}},
   'До востребования': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']}},
   'Краткосрочные': {'None': {'None': ['Количество операций', 'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['К

### Подгрузка данных и разделение на тренировочную и валидационную выборки

In [4]:
queries_df = pd.read_csv('data/queries.csv')
queries_df.head(2)

,text,keywords,k_count
0,Дай сводку: для sme.,SME,1
1,где презентацИя вЧЕрАшняЯ леЖИт?,NaN,0


In [5]:
queries_df.fillna('None', inplace=True)

In [6]:
X, y = queries_df.drop('k_count', axis=1), queries_df['k_count']

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, stratify=y, random_state=42)

In [7]:
X_val.shape[0]

240

In [8]:
y_train.value_counts(normalize=True)

k_count
4    0.143156
1    0.143156
5    0.143156
2    0.143156
0    0.143156
6    0.142111
3    0.142111
Name: proportion, dtype: float64

In [9]:
y_val.value_counts(normalize=True)

k_count
3    0.145833
6    0.145833
1    0.141667
5    0.141667
0    0.141667
4    0.141667
2    0.141667
Name: proportion, dtype: float64

In [10]:
y_train = pd.DataFrame({'keywords': X_train['keywords'], 'k_count': y_train})
y_val = pd.DataFrame({'keywords': X_val['keywords'], 'k_count': y_val})

In [11]:
X_train.drop('keywords', axis=1, inplace=True)
X_val.drop('keywords', axis=1, inplace=True)

# 3. Описание алгоритма решения

### 1. fit_hierarchy

Сперва вытаскиваем из нашего дерева нормализованные ключевые слова, а также словарь типа "нормализованнео значение - ключ в исходнике". Так мы быстро можем собирать и валидировать ответ

### 2. process_query. Общее описание

Обработка запроса. Разделяется на два алгоритма: первый - long_fuzzy_match - обрабатывает ключи с двумя и более словами с помощью вложенных окон; второй - short_fuzzy_match - обрабатывает слова по одному, тем самым идеально подходит для однословных ключей.
<br><br>
После того, как алгоритмы вернули свои слова, они проходят проверку по контекстной матрице. Если проверка не пройдена, то запускается проверка по схожим словам (этот алгоритм решает проблему сильно похожих слов на разных ветках). В конце снова проверяется контекстная совместимость. 
<br><br>
На выход подается словарь с найденными значениями и уровнями, на которых нашлись эти значения.

### 2.1. long_fuzzy_match

Обработка ключей с двумя и более словами. Сначала по окну размером 5 выбираются кандидаты, которые вероятнее всего присутствуют. Для точности используется функция из библиотеки rapidfuzz fuzz.partial_token_set_ratio, которая дает высокий показатель схожести, если словосочетания имеют хотя бы какие-то похожие части. Для уточнения результата по первому окну проходимся окном меньшего размера с более точной функцией - fuzz.token_set_ratio.

### 2.2. short_fuzzy_match

Обработка однословных ключей. Обычный мэтч по словам. Остановился на jaro_winkler, т.к. дает высокий показатель для опечаток и перестановок букв слов.

### 2.3. update_similarity_matrix

Функция, наполняющая матрицу схожести слов. Схожесть вычисляем по расстоянию Дамерау-Левенштейна.

### 2.4. update_context_matrix

Функция, наполняющая контекстную матрицу. В нашем случае, считаем, что слова контекстно совместимы, если любое слово является потомком или родителем остальных (т.е. не допускаются слова с одного уровня).

### 2.5. build_path

Функция, составляющая запрос в виде словаря по уровням.

### 3. get_dynamic_query_list

Функция, возвращающая список из подходящих значений. Следует следующим требованиям:

1. Если какие-то значения пропущены, то берем первое значение не None, считая справа, и пытаемся восстановить всевозможные пути/комбинации до него.
2. Если не выбран драйвер, то выводятся оба списка.

### Плюсы алгоритма:

1. Работает быстрее LLM и тратит меньше ресурсов.
2. Настраиваемый под разные иерархии.
3. Настраиваемый под любую точность.
4. Обрабатывает случаи типа "агентские" - "агентские обязанности", "онлайн" - "офлайн" и т.д.
5. Справляется с любым порядком слов, знаками препинания и регистром.
6. Обнаруживает контекстные конфликты
7. Отлавливает "лишние" слова на мэтчах.

### Минусы алгоритма:

1. Требует настройки.
2. Ломается, если в иерархии два абсолютно одинаковых ключа.
3. Дилемма опечаток-лишних слов: если поставить низкий порог схожести для мэтчей и матрицы схожести, то в запрос включаются лишние слова, если поставить слишком высокий - даже формы слова перестают мэтчиться.
4. Не успел дописать поддержку синонимов через ;

### Решения проблема:

1. Подбор гиперпараметров под свою иерархию.
2. Как-то индексировать ключи и проводить поиск по ним, сверяться с контекстом.
3. Использовать в качестве синонимов все формы слов.

# 4. ShowCase - демонстрация: метрики, тестирование, UI

### Модельный вид решения

In [18]:
from utils.final_model import *

model = KeywordsQueryProcessor()
model.fit(hierarchy, levels)

model.transform('Что там по первичке ипотека?')


CHOSEN TERMS: ['первичка', 'ипотека']

FINAL TERMS: ['первичка', 'ипотека']


{'segment': 'None',
 'lvl_1': 'None',
 'lvl_2': 'Ипотека',
 'lvl_3': 'Первичка',
 'lvl_4': 'None',
 'driver_1': 'None'}

### Демонстрация UI

![](./data/screen1.png)

![](./data/screen2.png)

![](./data/screen3.png)

![](./data/screen4.png)

![](./data/screen5.png)

![](./data/screen6.png)

### Метрики:

* "Грубый" accuracy - правильно ли или неправильно составил запрос.
* By-word accuracy - доля правильно угаданных слов.
<br><br>
Метрики будем исследовать в разрезе количества ключевых слов, а также длин предложений.

In [ ]:
accuracy_dct = {0: [], 1: [], 2: [], 3: [], 4: [], 5: []}

for query, key_words in queries.items():
    processed_query = process_query(query, all_terms, norm2keys, hierarchy, levels, 
                                    # long_score_cutoff_first=75,
                                    # long_score_cutoff_second=85,
                                    long_score_cutoff_first=65,
                                    long_score_cutoff_second=81, 
                                    sim_threshold=80,
                                    short_score_cutoff=91,
                                    sim_scorer=fuzz.partial_token_set_ratio
                                    #long_scorer_second=fuzz.WRatio
                                    )
    
    if not key_words:
        accuracy_dct[0].append(int(processed_query is None))
        continue
    if not processed_query:
        accuracy_dct[len(key_words)].append(0)
        continue
    
    terms_selected = [q for q in processed_query.values() if q != 'None']
    terms_guessed = list(set([t for t in terms_selected if t in key_words]))
    accuracy_dct[len(key_words)].append(len(terms_guessed)/len(key_words))

print(accuracy_dct)

# 5. Итоги и результаты